In [1]:
!pip install transformers
!pip install datasets
!pip install seqeval
!pip install ipywidgets
!pip install IProgress

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import torch

In [3]:
import transformers
from datasets import load_dataset, load_metric
from datasets import Sequence, ClassLabel
from datasets import Value

In [4]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [5]:
# dataset = load_dataset("conll2003", revision="master")

In [6]:
# dataset

In [7]:
PE_DATA_FILE = '/notebooks/token_classification/data/pe_dataset_task_1.pt'

In [8]:
pe_dataset = torch.load(PE_DATA_FILE)

In [9]:
pe_dataset

DatasetDict({
    train: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags'],
        num_rows: 1091
    })
    test: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags'],
        num_rows: 355
    })
    validation: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags'],
        num_rows: 273
    })
})

In [72]:
labels_object = pe_dataset['train'].features['paragraph_tags'].feature

In [73]:
def create_int_tags(batch):
    # tokens = tokenizer(batch['text'], padding=True, truncation=True, max_length=128)
    tokens['paragraph_tags_int'] = labels_object.str2int(batch['paragraph_tags'])
    return tokens

In [74]:
pe_dataset = pe_dataset.map(create_int_tags, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

TypeError: unhashable type: 'list'

In [10]:
# dataset['train'].features

In [11]:
pe_dataset['train'].features

{'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_nr': Value(dtype='string', id=None),
 'para_nr': Value(dtype='int64', id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [12]:
pe_dataset['train'].features['paragraph_tags'] = Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None),
                                             length=-1, id=None)

In [13]:
pe_dataset['train'].features['paragraph_tags']

Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None), length=-1, id=None)

In [14]:
pe_dataset['test'].features['paragraph_tags'] = Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None),
                                             length=-1, id=None)

In [15]:
pe_dataset['validation'].features['paragraph_tags'] = Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None),
                                             length=-1, id=None)

In [16]:
pe_dataset['test'].features['paragraph_tags']

Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None), length=-1, id=None)

In [17]:
pe_dataset['validation'].features['paragraph_tags']

Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None), length=-1, id=None)

In [18]:
pe_dataset['train'].features

{'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_nr': Value(dtype='string', id=None),
 'para_nr': Value(dtype='int64', id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-MajorClaim', 'I-MajorClaim', 'B-Claim', 'I-Claim', 'B-Premise', 'I-Premise'], names_file=None, id=None), length=-1, id=None)}

In [20]:
from transformers import AutoTokenizer

In [21]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [22]:
# def tokenize_and_align_labels(examples):
#     tokenized_inputs = tokenizer(examples["paragraph_words"], truncation=True, is_split_into_words=True)

#     labels = []
#     for i, label in enumerate(examples["paragraph_tags"]):
#         word_ids = tokenized_inputs.word_ids(batch_index=i)
#         previous_word_idx = None
#         label_ids = []
#         for word_idx in word_ids:
#             # Special tokens have a word id that is None. We set the label to -100 so they are automatically
#             # ignored in the loss function.
#             if word_idx is None:
#                 label_ids.append(-100)
#             # We set the label for the first token of each word.
#             elif word_idx != previous_word_idx:
#                 label_ids.append(label[word_idx])
#             # For the other tokens in a word, we set the label to either the current label or -100, depending on
#             # the label_all_tokens flag.
#             else:
#                 label_ids.append(label[word_idx] if label_all_tokens else -100)
#             previous_word_idx = word_idx

#         labels.append(label_ids)

#     tokenized_inputs["labels"] = labels
#     return tokenized_inputs

In [51]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [68]:
def tokenize_and_align_labels_hf(examples):
    tokenized_inputs = tokenizer(
        examples["paragraph_words"], truncation=True, is_split_into_words=True
    )
    # all_labels = examples["paragraph_tags"]
    print(examples["paragraph_tags"])
    all_labels = labels_object.str2int(examples["paragraph_tags"])
    
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [69]:
label_all_tokens = True

In [70]:
#labels_object.str2int(['O', 'B-Premise'])

In [71]:
tokenized_datasets = pe_dataset.map(tokenize_and_align_labels_hf, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

[['O', 'O', 'B-Claim', 'I-Claim', 'I-Claim', 'I-Claim', 'I-Claim', 'I-Claim', 'I-Claim', 'I-Claim', 'I-Claim', 'I-Claim', 'I-Claim', 'I-Claim', 'I-Claim', 'I-Claim', 'O', 'O', 'O', 'O', 'O', 'B-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'O', 'B-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'O', 'O', 'O', 'O', 'O', 'B-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'O', 'O', 'O', 'O', 'B-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'I-Premise', 'O', 'O', 'B-Premise', 'I-Premise', 'I-Premise', 'I-Premi

TypeError: unhashable type: 'list'

In [54]:
label_list = pe_dataset["train"].features['paragraph_tags'].feature.names

In [55]:
label_list

['O',
 'B-MajorClaim',
 'I-MajorClaim',
 'B-Claim',
 'I-Claim',
 'B-Premise',
 'I-Premise']

In [56]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN t

In [57]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
          

In [58]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    #push_to_hub=True,
)

In [59]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [60]:
metric = load_metric("seqeval")

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

In [61]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [62]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [63]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, ner_tags, tokens, chunk_tags, pos_tags.
***** Running training *****
  Num examples = 14041
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2634
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Loss.cu:247: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
/opt/pytorch/pytorch/aten/src/ATen/native/cuda/Loss.cu:247: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [31,0,0] Assertion `t >= 0 && t < n_classes` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [34]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, tokens, pos_tags, ner_tags, chunk_tags. If id, tokens, pos_tags, ner_tags, chunk_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3251
  Batch size = 16


{'eval_loss': 0.062008101493120193,
 'eval_precision': 0.9235021003758568,
 'eval_recall': 0.9345564380803222,
 'eval_f1': 0.9289963858771199,
 'eval_accuracy': 0.9831286637965272,
 'eval_runtime': 4.7756,
 'eval_samples_per_second': 680.746,
 'eval_steps_per_second': 42.717,
 'epoch': 3.0}

In [35]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: id, tokens, pos_tags, ner_tags, chunk_tags. If id, tokens, pos_tags, ner_tags, chunk_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3251
  Batch size = 16


{'CL': {'precision': 0.9753127057274523,
  'recall': 0.9765985497692815,
  'f1': 0.9759552042160738,
  'number': 3034},
 'LOC': {'precision': 0.9522727272727273,
  'recall': 0.9602750190985485,
  'f1': 0.9562571319893495,
  'number': 2618},
 'MISC': {'precision': 0.7924089852827265,
  'recall': 0.8310316815597075,
  'f1': 0.8112609040444091,
  'number': 1231},
 'ORG': {'precision': 0.8926336061627347,
  'recall': 0.9017509727626459,
  'f1': 0.8971691265424632,
  'number': 2056},
 'overall_precision': 0.9235021003758568,
 'overall_recall': 0.9345564380803222,
 'overall_f1': 0.9289963858771199,
 'overall_accuracy': 0.9831286637965272}